In [1]:
!pip install power-grid-model-ds[visualizer] --quiet


[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
from dataclasses import dataclass
from power_grid_model_ds import Grid, GraphContainer
from power_grid_model_ds.arrays import NodeArray, LineArray
from power_grid_model_ds.enums import NodeType

In [3]:
# 📦 Extend the grid with x, y, u (node) and i_from (line)
from numpy.typing import NDArray

class MyNodeArray(NodeArray):
    _defaults = {"x": 0.0, "y": 0.0, "u": 0.0}
    x: NDArray[np.float64]
    y: NDArray[np.float64]
    u: NDArray[np.float64]

class MyLineArray(LineArray):
    _defaults = {"i_from": 0.0}
    i_from: NDArray[np.float64]

    @property
    def is_overloaded(self) -> NDArray[np.bool_]:
        """Check if the line is overloaded."""
        return self.i_from > self.i_n

@dataclass
class MyGrid(Grid):
    node: MyNodeArray
    line: MyLineArray
    graphs: GraphContainer

In [4]:
from power_grid_model_ds.generators import RadialGridGenerator

# Generate a radial grid
grid_gen = RadialGridGenerator(grid_class=MyGrid, nr_nodes=20, nr_sources=1)
grid = grid_gen.run(seed=42)

np.random.seed(42)
num_nodes = grid.node.size
x_coords = np.random.randint(0, 1000, size=num_nodes)
y_coords = np.random.randint(0, 1000, size=num_nodes)

grid.node.x = x_coords
grid.node.y = y_coords

In [5]:
grid.sym_load.p_specified = np.random.randint(300000, 1200000, size=grid.sym_load.size)

In [6]:
grid.set_feeder_ids()
print(grid.node)

 id | u_rated | node_type | feeder_branch_id | feeder_node_id |   x   |   y   |  u  
 1  | 10500.0 |     0     |        81        |       61       | 102.0 | 308.0 | 0.0 
 2  | 10500.0 |     0     |        63        |       61       | 435.0 | 769.0 | 0.0 
 3  | 10500.0 |     0     |        63        |       61       | 860.0 | 343.0 | 0.0 
 4  | 10500.0 |     0     |        66        |       61       | 270.0 | 491.0 | 0.0 
 5  | 10500.0 |     0     |        73        |       61       | 106.0 | 413.0 | 0.0 
                                 (..11 hidden rows..)                                
 17 | 10500.0 |     0     |        63        |       61       |  99.0 | 856.0 | 0.0 
 18 | 10500.0 |     0     |        63        |       61       | 871.0 | 560.0 | 0.0 
 19 | 10500.0 |     0     |        73        |       61       | 663.0 | 474.0 | 0.0 
 20 | 10500.0 |     0     |        63        |       61       | 130.0 |  58.0 | 0.0 
 61 | 10500.0 |     1     |   -2147483648    |  -2147483648   | 

In [7]:
from power_grid_model_ds import PowerGridModelInterface

pgm_interface = PowerGridModelInterface(grid)
pgm_interface.calculate_power_flow()
pgm_interface.update_grid()


In [8]:
old_sub = grid.node.filter(node_type=NodeType.SUBSTATION_NODE)
print(old_sub)

 id | u_rated | node_type | feeder_branch_id | feeder_node_id |   x   |   y   |     u     
 61 | 10500.0 |     1     |   -2147483648    |  -2147483648   | 661.0 | 510.0 |10498.658..


In [9]:
# 🏗️ Add a new substation at (212, 133)
new_sub = MyNodeArray(
    node_type=[NodeType.SUBSTATION_NODE.value],
    u_rated=[10500.0],
    x=[212],
    y=[133]
)
grid.append(new_sub)
new_id = new_sub.id

print("\nAll node IDs and coordinates:")
for i in range(len(grid.node)):
    print(f"ID: {grid.node.id[i]} - ({grid.node.x[i]}, {grid.node.y[i]})")



All node IDs and coordinates:
ID: 1 - (102.0, 308.0)
ID: 2 - (435.0, 769.0)
ID: 3 - (860.0, 343.0)
ID: 4 - (270.0, 491.0)
ID: 5 - (106.0, 413.0)
ID: 6 - (71.0, 805.0)
ID: 7 - (700.0, 385.0)
ID: 8 - (20.0, 191.0)
ID: 9 - (614.0, 955.0)
ID: 10 - (121.0, 276.0)
ID: 11 - (466.0, 160.0)
ID: 12 - (214.0, 459.0)
ID: 13 - (330.0, 313.0)
ID: 14 - (458.0, 21.0)
ID: 15 - (87.0, 252.0)
ID: 16 - (372.0, 747.0)
ID: 17 - (99.0, 856.0)
ID: 18 - (871.0, 560.0)
ID: 19 - (663.0, 474.0)
ID: 20 - (130.0, 58.0)
ID: 61 - (661.0, 510.0)
ID: 93 - (212.0, 133.0)


In [ ]:
# visualize the grid
# TODO: Link some good first issues for the visualizer

from power_grid_model_ds.visualizer import visualize
visualize(grid)

In [11]:
# 📍 Identify a route with an overload
print(grid.line[grid.line.is_overloaded])

overloaded_line = grid.line[grid.line.is_overloaded][0]
overloaded_nodes = grid.node.filter(feeder_branch_id=overloaded_line.feeder_branch_id)

# 🔎 Find closest node in the overloaded route to the new substation
pos_new = np.array([new_sub.x, new_sub.y])
closest_node = min(
    overloaded_nodes.id,
    key=lambda nid: np.linalg.norm(pos_new - np.array([
        grid.node.x[grid.node.id == nid][0],
        grid.node.y[grid.node.id == nid][0]
    ]))
)
print(f"Closest node in overload path to new substation: {closest_node}")

 id | from_node | to_node | from_status | to_status | feeder_branch_id | feeder_node_id | is_feeder |   r1  |   x1  |  c1 | tan1 |   i_n   |  i_from 
 67 |     2     |    14   |      1      |     1     |        63        |       61       |   False   |0.180..|0.008..| 0.0 | 0.0  |163.035..|380.959..
 70 |     14    |    18   |      1      |     1     |        63        |       61       |   False   |0.077..|0.025..| 0.0 | 0.0  |237.064..|250.859..
Closest node in overload path to new substation: 10


In [13]:
# Add a connection from the new substation to its closest node
all_nodes_except_new = grid.node.id[grid.node.id != new_id]
distances = [
    (nid, np.linalg.norm(pos_new - np.array([
        grid.node.x[grid.node.id == nid][0],
        grid.node.y[grid.node.id == nid][0]
    ])))
    for nid in all_nodes_except_new
]

closest_existing_node = sorted(distances, key=lambda x: x[1])[0][0]

grid.append(MyLineArray(
    from_node=[new_id],
    to_node=[closest_existing_node],
    from_status=[1],
    to_status=[1],
    i_n=[200.0],
    r1=[0.05], x1=[0.01], c1=[0.0], tan1=[0.0]
))

print(f"Connected new substation to node {closest_existing_node}")


Connected new substation to node 10


In [14]:
# 🔗 Trace path from that node to the current (old) substation
path, length = grid.graphs.active_graph.get_shortest_path(closest_existing_node, old_sub.id[0])
print("Path from overload to old substation:", path)

Path from overload to old substation: [10, 17, 14, 2, np.int32(61)]


In [ ]:
# 🔪 Insert open point such that the load connected to old substation (node 1) is < 2_000_000W
cut_index = None

for i, node_id in enumerate(path[1:]):
    nodes_in_between = grid.graphs.active_graph.get_connected(2, nodes_to_ignore=[1, 3], inclusive=True)
    cumulative_load = grid.sym_load.filter(node=path[:i]).p_specified.sum()
    if cumulative_load > 2_000_000:
        cut_index = i - 1
        break

if cut_index is None:
    cut_index = len(path) - 2

nodes_to_cut = path[cut_index:cut_index + 2]


In [ ]:
line_to_open = grid.line.filter(from_node=nodes_to_cut, to_node=nodes_to_cut)
print(f"Line to open: \n{line_to_open}")
grid.make_inactive(line_to_open)

Line to open: 
 id | from_node | to_node | from_status | to_status | feeder_branch_id | feeder_node_id | is_feeder |  r1  |   x1  |  c1 | tan1 |   i_n   | i_from 
 67 |     2     |    14   |      1      |     1     |        63        |       61       |   False   |0.18..|0.008..| 0.0 | 0.0  |163.035..|380.95..


In [17]:
pgm_interface = PowerGridModelInterface(grid)
pgm_interface.calculate_power_flow()
pgm_interface.update_grid()

print(grid.line[grid.line.is_overloaded])

 id | from_node | to_node | from_status | to_status | feeder_branch_id | feeder_node_id | is_feeder | r1 | x1 | c1 | tan1 | i_n | i_from 

